In [ ]:
# Data is generated by the following command on one node: 
# nvidia-smi --query-gpu=timestamp,gpu_uuid,pstate,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used,power.draw,fan.speed --format=csv -l 10 |tee nvidia_4nodes.txt
# You can change the value for -l from 10 (sec) to 5 to collect data every 5 seconds.

In [ ]:
import pandas as pd

import plotly.graph_objects as go
from plotly.offline import plot

In [ ]:
# Change "nvidia_2nodes.txt2" to your file name.  The assumption is that the file is in the current directory

In [ ]:
df = pd.read_csv('nvidia_2nodes.txt2', sep=',')
df.columns = [x.strip() for x in df.columns]
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()

In [ ]:
df.dtypes

In [ ]:
# Change string to numeric

In [ ]:
df['utilization.gpu [%]'] = df['utilization.gpu [%]'].str.replace(' %', '').astype(int)
df['utilization.memory [%]'] = df['utilization.memory [%]'].str.replace(' %', '').astype(int)

for x in ['memory.total [MiB]', 'memory.free [MiB]', 'memory.used [MiB]']:
    tmp = df[x].copy()
    df[x] = tmp.str.strip().str.split(' ', expand=True)[0].astype(int)
    
df['power.draw [W]'] = df['power.draw [W]'].str.replace(' [W]', '').astype(float)

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
to_save = []
def plot_feature_by_gpu(y_axis, save=False):
    fig = go.Figure()
    for uuid in df['uuid'].unique():
        fig.add_trace(go.Scatter(x=df.loc[df['uuid']==uuid]['timestamp'],y=df.loc[df['uuid']==uuid][y_axis],name=uuid))

    fig.update_layout(title=y_axis)
    if save:
        to_save.append(plot(fig, config={"displayModeBar": False}, show_link=False,
                            include_plotlyjs=False, output_type='div'))
    fig.show()

In [ ]:
def plot_util_by_gpu_hist(y_axis, save=False):
    fig = go.Figure()
    for uuid in df['uuid'].unique():
        fig.add_trace(go.Histogram(x=df.loc[df['uuid']==uuid][y_axis], 
                                   histnorm='percent',
                                   name=uuid,
                                   xbins=dict(size=10)
                                  ))
    fig.update_layout(xaxis_title=y_axis, yaxis_title='Percentage', title=f'{y_axis} distribution')
    if save:
        to_save.append(plot(fig, config={"displayModeBar": False}, show_link=False,
                            include_plotlyjs=False, output_type='div'))
    fig.show()

In [ ]:
def plot_util_by_gpu_hist(y_axis, save=False):
    fig = go.Figure()
    for uuid in df['uuid'].unique():
        fig.add_trace(go.Histogram(x=df.loc[df['uuid']==uuid][y_axis], 
                                   histnorm='percent',
                                   name=uuid,
                                   xbins=dict(size=10)
                                  ))
    fig.update_layout(xaxis_title=y_axis, yaxis_title='Percentage', title=f'{y_axis} distribution')
    if save:
        to_save.append(plot(fig, config={"displayModeBar": False}, show_link=False,
                            include_plotlyjs=False, output_type='div'))
    fig.show()

In [ ]:
cols = df.columns.tolist()
for col in cols[3:-1]:
    plot_feature_by_gpu(col, True)
    if col.startswith('utilization'):
        plot_util_by_gpu_hist(col, save=True)

In [ ]:
# Export all plots to one html page
# Change title, head, and html file name accordingly

# html head and body start

In [ ]:
html_pre = '''
<html>
    <head>
      <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
      <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
      <style>body{ margin:100 100; background:white; }</style>
      <title>nvidia-smi profiling results</title>
    </head>
    <body>
      <h2 align="center">nvidia-smi profiling results</h2>
'''

# Plots
h3_string = ''

for p in to_save:
    h3_string += '''
      <hr>
      <h3></h3>
      ''' + p + '''
    '''

# 
html_post = '''
    </body>
</html>'''

html_string = html_pre + h3_string + html_post
html_name = 'nvidia-smi_profiling_results_demo.html'
with open(html_name, 'w') as f:
    f.write(html_string)